In [ ]:
# library(glmnet)

In [ ]:
crime_data <- read.table("./data 11.1/uscrime.txt", header = TRUE)

In [ ]:
head(crime_data)

In [ ]:
model <- lm(Crime~., data = crime_data)
summary(model)

#### R-squared is 0.8031 and adjusted R-squared is 0.7078

In [ ]:
# Stepwise Regression
stepwise_model <- step(lm(Crime ~ ., data = crime_data))

In [ ]:
# Summary of the model
summary(stepwise_model)

#### With the new model after performing stepwise regression we get a slightly lower R-squared value of 0.7888, but a slightly higher adjusted R-squared value of 0.7444. 

In [ ]:

uscrime_scale <- cbind(as.data.frame(scale(crime_data[,1])), 
                      as.data.frame(crime_data[,2]), 
                      as.data.frame(scale(crime_data[,c(3,4,5,6,7,8,9,10,11,12,13,14,15)])),
                      as.data.frame(crime_data[,16]))
 
# Fix the column names
colnames(uscrime_scale) <- colnames(crime_data)
# Display scaled data
head(uscrime_scale)

In [ ]:
# Run lasso 
uscrime_lasso <- cv.glmnet(x=as.matrix(uscrime_scale[,-16]), y=as.matrix(uscrime_scale$Crime), 
                      alpha=1, nfolds = 5, type.measure="mse", family="gaussian")

# Find the lamba with the smallest cvm
x <- uscrime_lasso$cvm
which(x == min(x))

In [ ]:
# The lamba for the smallest cvm
uscrime_lasso$lambda[which(x == min(x))]

In [ ]:
# or
uscrime_lasso$lambda.min

In [ ]:
coefficients(uscrime_lasso, s=uscrime_lasso$lambda.min)


In [ ]:
# Now lets make a model with these coefficients 
uscrime_lasso_lm <- lm(Crime ~M+So+Ed+Po1+M.F+Pop+NW+U1+U2+Wealth+Ineq+Prob, 
                    data = uscrime_scale)
summary(uscrime_lasso_lm)

In [ ]:
# Let's remove factor which p>0.05 and show the new model. 
uscrime_lasso_lm2 <- lm(Crime ~M+ Ed+ Po1+ U2+ Ineq+Prob, data = uscrime_scale)
summary(uscrime_lasso_lm2)

In [ ]:
# Elastic net model

r2=c()
for (i in 0:10) {
  mod_uscrime_elastic <- cv.glmnet(x=as.matrix(uscrime_scale[,-16]),y=as.matrix(uscrime_scale$Crime),
                            alpha=i/10, nfolds = 5, type.measure="mse",
                            family="gaussian")
  #dev.ratio is the percentage of deviance explained
  #min index for the dev.ratio of the model
  r2 = cbind(r2, mod_uscrime_elastic$glmnet.fit$dev.ratio[which(mod_uscrime_elastic$glmnet.fit$lambda == mod_uscrime_elastic$lambda.min)])
}

#get the best alpha
alpha <- (which.max(r2)-1)/10


#get model with alphas = 0.05
uscrime_elastic <- cv.glmnet(x=as.matrix(uscrime_scale[,-16]), 
                             y=as.matrix(uscrime_scale$Crime), alpha=0.05, 
                             nfolds = 5, type.measure="mse", family="gaussian")

uscrime_elastic_lm = lm(Crime ~M+So+Ed+Po1+Po2+LF+M.F+NW+U1+U2+Wealth+Ineq+Prob
+Time, data = uscrime_scale)
summary(uscrime_elastic_lm)

In [ ]:
uscrime_elastic_lm2 = lm(Crime ~M+Ed+U2+Ineq+Prob, data = uscrime_scale)
summary(uscrime_elastic_lm2)